In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} 3D visualization of Riemann sheets
TR-999
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=4) of the PDG.
:::
::::

# Riemann sheets

<!-- cspell:disable -->

In [ ]:
%pip install -q ampform==0.14.6 plotly==5.17.0 sympy==1.12

First, we formulate the $T$&nbsp;matrix in terms of a $K$&nbsp;matrix. There are two ways to do this and we associate the one with the $+$ with **Sheet I** and the one with $-$ with **Sheet II**.

In [ ]:
from __future__ import annotations

import warnings

import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import UnevaluatedExpression, implement_doit_method
from ampform.sympy.math import create_expression
from IPython.display import Math

warnings.filterwarnings("ignore")

Note that this the above inversion is equivalent to [Eq.&nbsp;(50.31)](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=4):

$$
T = (1 \pm iK\rho)^{-1}K.
$$

As an aside, we also define a special expression class for a square root where you can choose the sign for negative arguments. This can be used later for the phase space factor&nbsp;$\rho$.

In [ ]:
@implement_doit_method
class SignedSqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, z, **hints) -> SignedSqrt:
        return create_expression(cls, z, **hints)

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        return Rf"\sqrt[+]{{{z}}}"


@implement_doit_method
class PosArg(UnevaluatedExpression):
    is_commutative = True

    def __new__(cls, z, **hints) -> SignedSqrt:
        return create_expression(cls, z, **hints)

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg + 2 * sp.pi, sp.im(z) < 0),
            (arg, True),
        )

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        return Rf"\arg^+\left({z}\right)"


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

This gives us all the ingredients to formulate expressions for the parametrization of the matrix elements.
> **Note**
>  In the following example we assume that $l$=0 

## Define phasespacefactor for non-equal masses 

In [ ]:
@implement_doit_method
class NonEqualPhspFactor(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m1, m2, **hints) -> PhspFactor:
        return create_expression(cls, s, m1, m2, sign=+1, *hints)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((1 - ((m1 + m2) ** 2) / s) * (1 - (m1 - m2) ** 2 / s))

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        sign = "+"
        return Rf"\rho^{{{sign}}}\left({s}\right)"


s, m_a, m_b = sp.symbols("s m_a m_b")
phsp_nonequal_mass = NonEqualPhspFactor(s, m_a, m_b)
Math(aslatex({phsp_nonequal_mass: phsp_nonequal_mass.doit(deep=False)}))

## Chew-Mandelstamm function  

### Define break-up momentum 

In [ ]:
@implement_doit_method
class Breakup_Momentum_Signedsqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m_a, m_b, **hints) -> PhspFactor:
        return create_expression(cls, s, m_a, m_b, sign=+1, **hints)

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt(
            ((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (4 * s)) / sp.sqrt(s)
        )

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        sign = "+"
        return Rf"q^{{{sign}}}\left({s}\right)"


breakup_momentum = Breakup_Momentum_Signedsqrt(s, m_a, m_b)
Math(aslatex({breakup_momentum: breakup_momentum.doit(deep=False)}))

In [ ]:
def Chew_Mandelstamm(s, m1, m2):
    q_break = Breakup_Momentum_Signedsqrt(s, m1, m2)
    return (
        16
        / (sp.pi) ** 2
        * (
            sp.log(m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q_break * m1 * m2)
            - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
        )
    )


chew_mandelstamm = Chew_Mandelstamm(s, m_a, m_b)
Math(aslatex(chew_mandelstamm))

## Define parameters values for plotting 

## Add K Matrix definition   

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

No Blatt-Weisskopf-Formfactors because $l=0$

In [ ]:
gamma_R, m_R, w_R = sp.symbols("gamma_R m_R Gamma_R")
k_expr = (gamma_R**2 * m_R * w_R) / (s - m_R**2)

In [ ]:
rho_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: NonEqualPhspFactor(s, m_a, m_b),
}
Math(aslatex(rho_expressions))

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: Chew_Mandelstamm(s, m_a, m_b),
}
Math(aslatex(cm_expressions))

In [ ]:
T_rho_expr = T_explicit[0, 0].xreplace(rho_expressions)

T_cm_expr = T_explicit[0, 0].xreplace(cm_expressions)

In [ ]:
T_cm_expr

In [ ]:
args = (s, m_a, m_b, m_R, w_R, gamma_R)
T_rho_func = sp.lambdify(args, T_rho_expr.doit())
T_cm_func = sp.lambdify(args, T_cm_expr.doit())

In [ ]:
x = np.linspace(0, 4, num=200)
y = np.linspace(1e-5, 1.5, num=100)
X, Yn = np.meshgrid(x, -y)
X, Yp = np.meshgrid(x, +y)
Zn = X + Yn * 1j
Zp = X + Yp * 1j
Tn = T_rho_func(Zn**2, 0.05, 0.04, 0.4, 0.1, 1.0)
Tp = T_rho_func(Zp**2, 0.05, 0.04, 0.4, 0.1, 1.0)

vmax = 0.3


def sty(t):
    return {
        "cmin": -vmax,
        "cmax": +vmax,
        "colorscale": "RdBu_r",
        "surfacecolor": t.imag,
    }


Sn = go.Surface(x=X, y=Yn, z=Tn.real, **sty(Tn), name="Unphysical")
Sp = go.Surface(
    x=X, y=Yp, z=Tp.real, **sty(Tp), name="Physical", colorbar_title="Re T"
)
y = Yp[0]
z = x + y * 1j
line = go.Scatter3d(
    x=x,
    y=y,
    z=T_rho_func(z**2, 0.05, 0.04, 0.4, 0.1, 1.0).real,
    marker={"size": 0},
    line={"color": "darkgreen", "width": 1},
)
fig = go.Figure(data=[Sn, Sp, line])
fig.update_layout(height=550, width=600)
fig.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)
fig.show()

Note that we matched the colors [of the PDG](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=4), but that we had to remap the sheet associations. As can be seen in the plotting code, we have the following associations:

|                      | Sheet I  | Sheet II |
|----------------------|----------|----------|
| $\mathrm{Im}(s) < 0$ | $T^{I}$  | $T^{II}$ |
| $\mathrm{Im}(s) > 0$ | $T^{II}$ | $T^{I}$  |

So the sheet numbers 'flip' for $\mathrm{Im}(s) > 0$ and what we see in the third figure is just $T^{II}$.